In [1]:
from vars.week_3 import *
from vars.week_4 import *
from vars.week_5 import *
from vars.week_6 import *

/home/kavi/.virtualenvs/aim-lab-sessions/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

In [3]:
class PlResNet(pl.LightningModule):
    def __init__(self, in_channels, outputs):
        super().__init__()
        # model arch goes here
        self.model = ResNet(in_channels, ResBlock, outputs)

    def forward(self, x):
        return self.model.forward(x)
    
    def configure_optimizers(self):
        return SGD(self.parameters(), lr=learning_rate)
    
    def training_step(self, batch, batch_idx):
        image_batch, label_batch = batch
        batch_sz = len(image_batch)
        image_batch = image_batch.reshape(batch_sz, 1, 28, 28) 
        output = model(image_batch)
        loss = nn.CrossEntropyLoss()(output, label_batch)
        
        preds = torch.argmax(output, dim=1)
        correct = int(torch.eq(preds, label_batch).sum())
        minibatch_acc = 100 * correct / batch_sz
        
        self.log("train_acc", minibatch_acc)
        self.log("train_loss", loss)
        return loss
    

    def validation_step(self, batch, batch_idx):
        image_batch, label_batch = batch
        batch_sz = len(image_batch)
        image_batch = image_batch.reshape(batch_sz, 1, 28, 28) 
        output = model(image_batch)
        loss = nn.CrossEntropyLoss()(output, label_batch)
        
        preds = torch.argmax(output, dim=1)
        correct = int(torch.eq(preds, label_batch).sum())
        minibatch_acc = 100 * correct / batch_sz
        
        self.log('val_loss', loss)
        self.log('val_acc', minibatch_acc)
        return loss
    

# load data
train_dl, val_dl, test_dl = load_data(DATA_PATH, batch_sz=batch_sz)

model = PlResNet(in_channels=1, outputs=10)

# define callbacks
# saves last-K checkpoints based on "global_step" metric
# make sure you log it inside your LightningModule
checkpoint_callback = ModelCheckpoint(save_top_k=5,
                                      monitor="val_loss",
                                      mode="min",
                                      dirpath="./saved_models",
                                      filename="mnist-{epoch:02d}-{val_loss:.2f}")
early_stop_callback = EarlyStopping(monitor="val_acc", 
                                    min_delta=0.00, 
                                    patience=3, 
                                    verbose=False, 
                                    mode="max")

# train
trainer = pl.Trainer(accelerator='gpu', 
                     devices=1, 
                     precision=16, 
                     limit_train_batches=0.5, 
                     callbacks=[early_stop_callback,
                                checkpoint_callback]) 
trainer.fit(model, train_dl, val_dl)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/kavi/.virtualenvs/aim-lab-sessions/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
/home/kavi/.virtualenvs/aim-lab-sessions/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:606: UserWarning: Checkpoint directory /home/kavi/Desktop/phd/teaching/aim-lab-sessions/saved_models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
11.2 M    Trainable params
0         Non-trainable par

Sanity Checking: 0it [00:00, ?it/s]

/home/kavi/.virtualenvs/aim-lab-sessions/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Epoch 0:  43%|██████████████████████████████████████████████████████████▏                                                                             | 197/460 [00:05<00:07, 34.53it/s, loss=0.187, v_num=3]
Validation: 0it [00:00, ?it/s]
Epoch 0:  51%|████████████████████████████████████████████████████████████████████▉                                                                   | 233/460 [00:06<00:06, 35.43it/s, loss=0.187, v_num=3]


Epoch 0:  59%|████████████████████████████████████████████████████████████████████████████████▍                                                       | 272/460 [00:07<00:04, 38.28it/s, loss=0.187, v_num=3]


...

Epoch 11:  67%|█████████████████████████████████████████████████████████████████████████████████████████▋                                           | 310/460 [00:07<00:03, 39.44it/s, loss=0.00654, v_num=3]


Epoch 11:  76%|████████████████████████████████████████████████████████████████████████████████████████████████████▉                                | 349/460 [00:08<00:02, 42.29it/s, loss=0.00654, v_num=3]


Epoch 11:  84%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                    | 388/460 [00:08<00:01, 44.80it/s, loss=0.00654, v_num=3]


Epoch 11:  93%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍         | 427/460 [00:09<00:00, 47.23it/s, loss=0.00654, v_num=3]


Epoch 11: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 460/460 [00:09<00:00, 48.63it/s, loss=0.00654, v_num=3]
